In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load training data, sort by id
train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv').sort_values(by='sig_id')
columns = train.columns
train.head()

In [ ]:
#Load testing data
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test.head()

In [ ]:
#drop id
train = train[columns[1:]]

#OHE categorical features
train_cat = train[columns[1:4]]
train.drop(columns[1:4], axis=1, inplace=True)
train_cat.head()

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

names = train_cat['cp_type'].value_counts().index
vals = train_cat['cp_type'].value_counts()
ax1.bar(names, vals)
ax1.set_title('cp_type')
ax1.grid()

names = list(map(lambda x: str(x), train_cat['cp_time'].value_counts().index))
vals = train_cat['cp_time'].value_counts()
ax2.bar(names, vals)
ax2.set_title('cp_time')
ax2.grid()

names = train_cat['cp_dose'].value_counts().index
vals = train_cat['cp_dose'].value_counts()
ax3.bar(names, vals)
ax3.set_title('cp_dose')
ax3.grid()

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(15, 15))
step = 10

for  i in range(0, 9, 1):
    row, col = i // 3, i % 3
    features = train.loc[:, 'c-{}'.format(i * step)]
    sns.kdeplot(features, ax=axes[row, col])
    axes[row, col].grid()
    axes[row, col].set_title('c-{}'.format(i * step))
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(15, 15))
step = 10

for  i in range(0, 9, 1):
    row, col = i // 3, i % 3
    features = train.loc[:, 'g-{}'.format(i * step)]
    sns.kdeplot(features, ax=axes[row, col])
    axes[row, col].grid()
    axes[row, col].set_title('g-{}'.format(i * step))
plt.show()

In [ ]:
rand_cell_feats = list(map(lambda x: 'c-{}'.format(x), np.random.randint(low=0, high=100, size=15)))
rand_gene_feats = list(map(lambda x: 'c-{}'.format(x), np.random.randint(low=0, high=100, size=15)))

plt.figure(figsize=(13, 8))
sns.heatmap(train[rand_cell_feats].corr())

plt.title('Correlation matrix for cell features', fontsize=20);

In [ ]:
rand_cell_feats = list(map(lambda x: 'g-{}'.format(x), np.random.randint(low=0, high=100, size=15)))
rand_gene_feats = list(map(lambda x: 'g-{}'.format(x), np.random.randint(low=0, high=100, size=15)))

plt.figure(figsize=(13, 8))
sns.heatmap(train[rand_cell_feats].corr())

plt.title('Correlation matrix for gene features', fontsize=20);

In [ ]:
#encode
encoder = OneHotEncoder(categories='auto')
train_cat_OHE = encoder.fit_transform(train_cat).toarray()
train_OHE = pd.concat((train, pd.DataFrame(train_cat_OHE)), axis=1)

In [ ]:
#Same One-Hot encoding
test_id = test[columns[0]].copy()
test = test[columns[1:]]

test_cat = test[columns[1:4]]
test.drop(columns[1:4], axis=1, inplace=True)

test_cat_OHE = encoder.transform(test_cat).toarray()
test_OHE = pd.concat((test, pd.DataFrame(test_cat_OHE)), axis=1)

In [ ]:
#Load training target data, sort by id
train_target_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv').sort_values(by='sig_id')
target_columns = train_target_scored.columns
train_target_scored.head()

In [ ]:
print("X_train shape: ", np.array(train_OHE).shape)
print("Y_train shape: ", np.array(train_target_scored[target_columns[1:]]).shape)

Видим, что значения признаков лежат на отрезке [-10, 10]

In [ ]:
fig = plt.figure(figsize=(8, 6))

print("Min: ", np.array(train_OHE).min(), "\nMax: ", np.array(train_OHE).max())

plt.plot(np.array(train_OHE)[0])
plt.show()

In [ ]:
Y = np.array(train_target_scored[target_columns[1:]])
target_num_features = Y.shape[1]

for f in range(target_num_features):
    print(f, ': ', (Y[:, f] != 0.0).sum())

In [ ]:
np.unique(Y)

In [ ]:
print("All zero objects portion: ", (Y.sum(axis=1) == 0).sum()/Y.shape[0])
print("One 1.0 value portion: ", (Y.sum(axis=1) == 1).sum()/Y.shape[0])
print("More than one 1.0 values portion: ", (Y.sum(axis=1) > 1).sum()/Y.shape[0])

In [ ]:
vals, counts = np.unique(Y.sum(axis=1), return_counts=True)

fig = plt.figure(figsize=(10, 8))
plt.bar(vals, counts)
plt.title("Counts: " + str(counts))
plt.grid()
plt.show()

In [ ]:
train_target = train_target_scored[target_columns[1:]]
train_target['Dummy'] = [0] * train_target.shape[0]

dummy_index = train_target.columns.get_loc("Dummy")
print("Dummy col num: ", dummy_index)

fill_dummy_mask = np.array(train_target).sum(axis=1) == 0
train_target.loc[fill_dummy_mask, 'Dummy'] = 1.0

In [ ]:
Y = np.array(train_target)
print("All zero objects portion: ", (Y.sum(axis=1) == 0).sum()/Y.shape[0])
print("One 1.0 value portion: ", (Y.sum(axis=1) == 1).sum()/Y.shape[0])
print("More than one 1.0 values portion: ", (Y.sum(axis=1) > 1).sum()/Y.shape[0])

In [ ]:
n_comp = 680
pca = PCA(n_components=n_comp, random_state=228)
pca.fit(np.array(pd.concat((train_OHE, test_OHE), axis=0)))

In [ ]:
train_OHE_pca = pca.transform(np.array(train_OHE))
train_OHE_aug = pd.concat((train_OHE, pd.DataFrame(train_OHE_pca)), axis=1)
train_OHE_aug.head()

In [ ]:
class Net(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Net, self).__init__()
        assert (num_features > num_targets), 'Neg diff'

        self.batch_norm1 = torch.nn.BatchNorm1d(num_features)
        self.linear1 = nn.utils.weight_norm(nn.Linear(num_features, 1024))
        
        self.batch_norm2 = torch.nn.BatchNorm1d(1024)
        self.linear2 = nn.utils.weight_norm(nn.Linear(1024, 1024))
        
        self.batch_norm3 = torch.nn.BatchNorm1d(1024)
        self.linear3 = nn.utils.weight_norm(nn.Linear(1024, num_targets))
        
        self.dropout = nn.Dropout(p=0.5)
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = F.elu(x)
        
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = F.elu(x)
        
        x = self.batch_norm3(x)
        x = self.dropout(x)
        x = self.linear3(x)
        
        return torch.sigmoid(x)

In [ ]:
X_train = np.array(train_OHE_aug)
Y_train = np.array(train_target)

smoothing = 0.0000
Y_train = Y_train.clip(min=smoothing, max=1.0-smoothing)



#p_num = 0
#for p in model.parameters():
#    p_num += np.prod(p.shape)
    
#print("Number of net parameters: ", p_num)

assert (X_train.shape[0] == Y_train.shape[0]), "Wrong train shapes"

epochs = 50
batch_size = 128

random_states = [42, 228, 1337, 2020]

for n_try, random_state in enumerate(random_states):

    n_splits = 7
    print("==Random State: ", random_state)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold_num, (train_ind, test_ind) in enumerate(kf.split(Y_train)):

        X_kf_train, Y_kf_train = X_train[train_ind], Y_train[train_ind]
        X_kf_test, Y_kf_test = X_train[test_ind], Y_train[test_ind]
        train_iters = X_kf_train.shape[0]//batch_size
        test_iters = X_kf_test.shape[0]//batch_size

        model = Net(num_features=X_train.shape[1], num_targets=Y_train.shape[1]).cuda()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        criterion = nn.BCELoss().cuda()
        best_epoch_loss = 1e18

        for epoch_num in range(1, epochs+1):
            train_loss = 0
            test_loss = 0

            for iter_num in range(train_iters):
                optimizer.zero_grad()
                batch_x = torch.tensor(X_kf_train[iter_num*batch_size:(iter_num+1)*batch_size]).float().cuda()
                batch_y = torch.tensor(Y_kf_train[iter_num*batch_size:(iter_num+1)*batch_size]).float().cuda()

                pred = model(batch_x)
                loss = criterion(pred, batch_y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            model.eval()
            with torch.no_grad():
                for iter_num_test in range(test_iters):
                    optimizer.zero_grad()
                    batch_x = torch.tensor(X_kf_test[iter_num_test*batch_size:(iter_num_test+1)*batch_size]).float().cuda()
                    batch_y = torch.tensor(Y_kf_test[iter_num_test*batch_size:(iter_num_test+1)*batch_size]).float().cuda()

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)
                    test_loss += loss.item()
            model.train()

            train_loss /= train_iters
            test_loss /= test_iters
            scheduler.step()


            print("N_try: ", n_try+1, "|| Fold num: ", fold_num+1, "|| Epoch num:", epoch_num, "|| Train Loss: ", train_loss, "|| Test Loss: ", test_loss)

            if test_loss < best_epoch_loss:
                best_epoch_loss = test_loss
                model.eval()
                torch.save(model.state_dict(), 'best_model_' + str(fold_num+1) + '_' + str(n_try+1) + '.pth')
                model.train()
                print("Best model so far")


In [ ]:
#Test prediction

test_OHE_pca = pca.transform(np.array(test_OHE))
test_OHE_aug = pd.concat((test_OHE, pd.DataFrame(test_OHE_pca)), axis=1)

test_prediction = 0
for n_try, random_state in enumerate(random_states):
    for model_num in range(1, n_splits+1):

        model = Net(num_features=X_train.shape[1], num_targets=Y_train.shape[1]).cuda()
        model.load_state_dict(torch.load('best_model_' + str(model_num) + '_' + str(n_try+1) + '.pth'))
        model.eval()
        with torch.no_grad():
            X_test = torch.tensor(np.array(test_OHE_aug)).float().cuda()
            test_prediction += model(X_test).cpu().numpy()

test_prediction /= (n_splits*len(random_states))
submission = pd.DataFrame(test_prediction[:, :-1], columns=target_columns[1:])


In [ ]:
test_id = pd.DataFrame(test_id, columns=[target_columns[0]])
submission = pd.concat((test_id, submission), axis=1)
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=None)